<a href="https://colab.research.google.com/github/Mayank187/StackoverflowTags/blob/main/stackoverflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle
!pip install scikit-multilearn
!pip install matplotlib --upgrade
!pip install category_encoders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.4/89.4 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 85.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.3/98.3 kB 10.8 MB/s eta 0:00:00
  Attempting uninstall: pyparsing
    Found existing installation: pyparsing 3.1.0
    Uninstalling pyparsing-3.1.0:
      Successfully uninstalled pyparsing-3.1.0
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.7.1
    Uninstalling matplotlib-3.7.1:
      Successfully uninstalled matplotlib-3.7.1


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 4.5 MB/s eta 0:00:00


In [2]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle competitions download facebook-recruiting-iii-keyword-extraction

100% 2.90G/2.90G [01:41<00:00, 34.3MB/s]
100% 2.90G/2.90G [01:41<00:00, 30.6MB/s]


In [ ]:
!unzip facebook-recruiting-iii-keyword-extraction.zip

Archive:  facebook-recruiting-iii-keyword-extraction.zip
  inflating: SampleSubmission.csv    
  inflating: Test.zip                

In [ ]:
!unzip Train.zip

In [ ]:
!rm Train.zip
!rm Test.zip
!rm SampleSubmission.csv
!rm facebook-recruiting-iii-keyword-extraction.zip

In [ ]:
def preprocess_text(text):
    # lowercasing
    text = text.lower()

    # Substituting multiple spaces with single space
    text = re.sub(r'\s+', ' ', text, flags=re.I)

    # Remove all special characters except '#' and '+'
    text = re.sub('[^a-zA-Z0-9 #+]', '', text)

    # tokenize the text
    words = word_tokenize(text)

    # remove stopwords and stem the words
    stop_words = set(stopwords.words('english'))
    # ps = PorterStemmer()
    ps = ps = SnowballStemmer(language='english')
    words = [ps.stem(w) for w in words if w not in stop_words]

    return ' '.join(words)

# Importing the Important Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk import WordNetLemmatizer
nltk.download('wordnet')
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import category_encoders as ce
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix, classification_report
from nltk.stem import PorterStemmer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report, hamming_loss, accuracy_score
from skmultilearn.problem_transform import BinaryRelevance, ClassifierChain, LabelPowerset
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn import metrics
from nltk.stem.snowball import SnowballStemmer

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')

In [ ]:
train_data = pd.read_csv('Train.csv',nrows=80000)

In [ ]:
train_data.head()

In [ ]:
train_data.shape

In [ ]:
train_data.info()

In [ ]:
train_data.describe(include='all').T

In [ ]:
train_data.isna().sum()

In [ ]:
train_data.dropna(inplace = True)

In [ ]:
train_data['Question'] = train_data.apply(lambda row: str(row['Title']) + str(row['Body']), axis=1)
train_data.drop(['Title','Body','Id'], axis=1, inplace=True)

In [ ]:
train_data.head()

In [ ]:
train_data['Question'] = train_data['Question'].apply(preprocess_text)

In [ ]:
train_data.head()

In [ ]:
# Convert text data into TF-IDF features
vectorizer = TfidfVectorizer(max_features=200000)
X = vectorizer.fit_transform(train_data['Question'])


cntvectorizer = CountVectorizer(tokenizer= lambda text : text.split(" "))
y = cntvectorizer.fit_transform(train_data['Tags'])

In [ ]:
cntvectorizer.vocabulary_

In [ ]:
print(X)

In [ ]:
print(y)

In [ ]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
print(X_train.shape)
print(y_train.shape)

In [ ]:
classifier = OneVsRestClassifier(SGDClassifier(loss='log_loss',alpha=0.00001, penalty='l1'), n_jobs=-1)
classifier.fit(X_train,y_train)

In [ ]:
predictions = classifier.predict(X_test)

In [ ]:
print("accuracy ", metrics.accuracy_score(y_test,predictions))
print("macro f1 score ",metrics.f1_score(y_test,predictions, average='macro'))
print("micro f1 score ", metrics.f1_score(y_test, predictions, average='micro'))
print("hamming loss ", metrics.hamming_loss(y_test,predictions))